In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Importing Libraries

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize

In [3]:
document = """Can you describe something about cricket?
Cricket is a bat-and-ball game that is played between two teams of eleven players on a field, at the centre of which is a 22-yard pitch with a wicket at each end, each comprising two bails balanced on three stumps.
What are the main formats of cricket?
Cricket is played mainly in three formats: Test cricket, One Day Internationals (ODIs), and Twenty20 (T20).
How long does a Test match last?
A Test match is the longest format of cricket and can last up to five days, with each team playing two innings.
How many overs are played in an ODI match?
An ODI match consists of 50 overs for each team.
How many overs are bowled in a T20 match?
A T20 match is a short format of cricket where each team plays 20 overs.
What are the ways a batter can be dismissed?
A batter can be out in several ways such as being bowled, caught, leg before wicket (LBW), run out, stumped, or hit wicket.
What is the term powerplay mean?
A powerplay is a phase of the match where fielding restrictions apply, allowing fewer fielders outside the 30-yard circle.
What do you mean by the term DRS system?
The Decision Review System (DRS) allows teams to challenge umpire decisions using technologies like Hawk-Eye and ball tracking.
What is a hat-trick in cricket?
A hat-trick occurs when a bowler takes three wickets in three consecutive deliveries.
What is a maiden over?
A maiden over is an over in which no runs are scored by the batting team.
Who manages international cricket?
International cricket is governed by the International Cricket Council (ICC), while each country has its own national cricket board.
How many countries play cricket?
Cricket is played in over 100 countries around the world, including 12 full member nations.
How many ICC trophies has India won?
India has won five major ICC trophies including two ODI World Cups, two T20 World Cups, and one Champions Trophy.
Who are some famous Indian cricketers?
Some of the most famous Indian cricketers include Sachin Tendulkar, MS Dhoni, Virat Kohli, Rohit Sharma, Rahul Dravid, and Jasprit Bumrah.
What are some records held by Virat Kohli?
Virat Kohli holds many records including most ODI centuries, fastest to multiple run milestones, and highest run-scorer in IPL history.
What is Virat Kohli known for in cricket?
He is known as the run machine due to his exceptional consistency, prolific run-scoring across all formats of cricket, and ability to perform under pressure.
What are some records of Rohit Sharma?
Rohit Sharma holds the record for the highest individual score in One Day Internationals with 264 runs. He is also the only player to have scored three double centuries in ODI cricket and has hit the most sixes in international cricket.
What is Rohit Sharma known for in limited-overs cricket?
Rohit Sharma is known for his explosive opening batting, ability to score big hundreds, and record-breaking performances in both ODIs and T20 Internationals.
How dominant has India been in Test cricket?
India has been one of the strongest Test teams in the world, especially at home, where they have remained unbeaten in Test series for more than a decade. Indian pitches combined with world-class spin bowling have made India extremely difficult to defeat.
Why is India considered strong in home Test matches?
India’s dominance at home is due to quality spinners, experienced batters, and the ability to adapt to subcontinent conditions, making visiting teams struggle.
What is the Indian Premier League (IPL)?
The Indian Premier League is a professional T20 cricket league in India where top players from around the world compete for franchise teams. It is one of the richest and most-watched cricket leagues globally.
Why is the IPL important in world cricket?
The IPL has revolutionized cricket by introducing fast-paced entertainment, high player salaries, and opportunities for young talent to play with international stars.
What is The Ashes?
The Ashes is a famous Test cricket series played between England and Australia. It is one of the oldest and most competitive rivalries in the sport.
Why is The Ashes so special?
The Ashes is special because of its long history, intense rivalry, and the high level of competition between two traditional cricketing nations.
What is the World Test Championship (WTC)?
The World Test Championship is an international tournament organized by the ICC to determine the best Test team in the world based on performances over a two-year cycle.
How does the World Test Championship work?
Teams earn points from Test series, and the top two teams qualify for the WTC final, where they compete to become Test world champions.
Has India performed well in the World Test Championship?
India has been one of the most consistent teams in the World Test Championship and has reached multiple finals due to strong performances at home and abroad.
Why is Test cricket still important in modern times?
Test cricket remains important because it tests a player’s skill, patience, and endurance over long periods, making it the ultimate format of the game."""

## Tokeinzation

In [4]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [5]:
tokens = word_tokenize(document.lower())

In [6]:
vocab = {'<uk>' : 0}

for token in Counter(tokens).keys():
    if token not in vocab:
        vocab[token] = (len(vocab))

vocab

{'<uk>': 0,
 'can': 1,
 'you': 2,
 'describe': 3,
 'something': 4,
 'about': 5,
 'cricket': 6,
 '?': 7,
 'is': 8,
 'a': 9,
 'bat-and-ball': 10,
 'game': 11,
 'that': 12,
 'played': 13,
 'between': 14,
 'two': 15,
 'teams': 16,
 'of': 17,
 'eleven': 18,
 'players': 19,
 'on': 20,
 'field': 21,
 ',': 22,
 'at': 23,
 'the': 24,
 'centre': 25,
 'which': 26,
 '22-yard': 27,
 'pitch': 28,
 'with': 29,
 'wicket': 30,
 'each': 31,
 'end': 32,
 'comprising': 33,
 'bails': 34,
 'balanced': 35,
 'three': 36,
 'stumps': 37,
 '.': 38,
 'what': 39,
 'are': 40,
 'main': 41,
 'formats': 42,
 'mainly': 43,
 'in': 44,
 ':': 45,
 'test': 46,
 'one': 47,
 'day': 48,
 'internationals': 49,
 '(': 50,
 'odis': 51,
 ')': 52,
 'and': 53,
 'twenty20': 54,
 't20': 55,
 'how': 56,
 'long': 57,
 'does': 58,
 'match': 59,
 'last': 60,
 'longest': 61,
 'format': 62,
 'up': 63,
 'to': 64,
 'five': 65,
 'days': 66,
 'team': 67,
 'playing': 68,
 'innings': 69,
 'many': 70,
 'overs': 71,
 'an': 72,
 'odi': 73,
 'consist

In [7]:
len(vocab)

342

## Convert Sentences into Indexes

In [8]:
#extract sentences
input_sentences = document.split('\n')

print(len(input_sentences))
input_sentences[0]

56


'Can you describe something about cricket?'

In [9]:
#function to convert text to indexes
def text_to_index(sentence, vocab):

    numerical_sentence = []
    
    for token in sentence :
        if token in vocab:
            numerical_sentence.append(vocab[token])
        else :
            numerical_sentence.append(vocab['<uk>'])
    return numerical_sentence

In [10]:
input_numerical_sentences = []

for sentence in input_sentences:
    input_numerical_sentences.append(text_to_index(word_tokenize(sentence.lower()), vocab))

In [11]:
len(input_numerical_sentences)

56

In [12]:
print(input_sentences[8])
print(input_numerical_sentences[8][:9])

How many overs are bowled in a T20 match?
[56, 70, 71, 40, 77, 44, 9, 55, 59]


## Create training sequence

If sentence is [3,5,1,2] then the training sequences will be [3,5] [3,5,1] [3,2,1,5]

In [13]:
training_sequence = []
for sentence in input_numerical_sentences:
    for i in range(1,len(sentence)) :
        training_sequence.append(sentence[:i+1])

len(training_sequence)

912

each sequence is of different length so we will apply padding in the begining

In [14]:
#length of longest sequence
len_longest_seq = len(max(training_sequence, key=len))
print(len_longest_seq)

46


In [15]:
padded_training_sequence = []
for sequence in training_sequence:
    padded_training_sequence.append([0]*(len_longest_seq-len(sequence)) + sequence)

In [16]:
print(f"Training sequence : {training_sequence[0]}, length : {len(training_sequence[0])}")
print(f"Padded sequence : {padded_training_sequence[0]}, length : {len(padded_training_sequence[0])}")

Training sequence : [1, 2], length : 2
Padded sequence : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2], length : 46


## Create Tensor

In [17]:
padded_training_seq = torch.tensor(padded_training_sequence, dtype=torch.long)
padded_training_seq

tensor([[ 0,  0,  0,  ...,  0,  1,  2],
        [ 0,  0,  0,  ...,  1,  2,  3],
        [ 0,  0,  0,  ...,  2,  3,  4],
        ...,
        [ 0,  0,  0,  ..., 62, 17, 24],
        [ 0,  0,  0,  ..., 17, 24, 11],
        [ 0,  0,  0,  ..., 24, 11, 38]])

In [18]:
x = padded_training_seq[:, :-1]
y = padded_training_seq[:,-1]

In [19]:
x

tensor([[  0,   0,   0,  ...,   0,   0,   1],
        [  0,   0,   0,  ...,   0,   1,   2],
        [  0,   0,   0,  ...,   1,   2,   3],
        ...,
        [  0,   0,   0,  ..., 341,  62,  17],
        [  0,   0,   0,  ...,  62,  17,  24],
        [  0,   0,   0,  ...,  17,  24,  11]])

In [20]:
y

tensor([  2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,   8,  13,  14,
         15,  16,  17,  18,  19,  20,   9,  21,  22,  23,  24,  25,  17,  26,
          8,   9,  27,  28,  29,   9,  30,  23,  31,  32,  22,  31,  33,  15,
         34,  35,  20,  36,  37,  38,  40,  24,  41,  42,  17,   6,   7,   8,
         13,  43,  44,  36,  42,  45,  46,   6,  22,  47,  48,  49,  50,  51,
         52,  22,  53,  54,  50,  55,  52,  38,  57,  58,   9,  46,  59,  60,
          7,  46,  59,   8,  24,  61,  62,  17,   6,  53,   1,  60,  63,  64,
         65,  66,  22,  29,  31,  67,  68,  15,  69,  38,  70,  71,  40,  13,
         44,  72,  73,  59,   7,  73,  59,  74,  17,  75,  71,  76,  31,  67,
         38,  70,  71,  40,  77,  44,   9,  55,  59,   7,  55,  59,   8,   9,
         78,  62,  17,   6,  79,  31,  67,  80,  81,  71,  38,  40,  24,  82,
          9,  83,   1,  84,  85,   7,  83,   1,  84,  86,  44,  87,  82,  88,
         89,  90,  77,  22,  91,  22,  92,  93,  30,  50,  94,  

## Creating Dataset and DataLoader classes

In [21]:
class CustomDataset(Dataset):
    def __init__(self,x,y):
        self.x = x
        self.y = y

    def __len__(self):
        return self.x.shape[0]

    def __getitem__(self,idx):
        return self.x[idx], self.y[idx]

In [22]:
#create object of customdataset
dataset = CustomDataset(x,y)

print(len(dataset))
print(dataset[0])

912
(tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]), tensor(2))


In [23]:
#create dataloader object
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

for input,output in dataloader:
    print(input,output)

tensor([[  0,   0,   0,  ..., 231, 232,  44],
        [  0,   0,   0,  ...,  13,  44, 137],
        [  0,   0,   0,  ...,   0,   0, 142],
        ...,
        [  0,   0,   0,  ..., 269, 279,  24],
        [  0,   0,   0,  ...,   0,  39,   8],
        [  0,   0,   0,  ...,  24,  95, 203]]) tensor([233, 157, 143, 275,  10,   6, 206, 151, 312, 161, 169, 207, 322, 160,
         20,  64,  92,  44,  23,   6, 175,  84,  24, 190, 314,  37,  50, 309,
        193, 341,  24, 204])
tensor([[  0,   0,   0,  ...,  44,  73,   6],
        [  0,   0,   0,  ...,  24, 110, 111],
        [  0,   0,   0,  ..., 183, 201,  76],
        ...,
        [  0,   0,   0,  ..., 113, 286, 287],
        [  0,   0,   0,  ...,   0,   0, 166],
        [  0,   0,   0,  ..., 284,  44, 333]]) tensor([ 53,  38,  44,   9,   6,  15,  79, 125,  38,  88,  22,   7,   7, 274,
         22, 119,  24, 192, 169,  22,  60,   8,  17,  22,  17,  48, 236,  44,
          6, 288, 150, 334])
tensor([[  0,   0,   0,  ..., 285,   6, 113],
    

## Create Model 

In [38]:
class LSTMModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, 100)  #100 : size of embedding vector
        self.lstm = nn.LSTM(100, 150, batch_first=True)
        self.fc = nn.Linear(150,vocab_size)

    def forward(self, x):
        embedded = self.embedding(x)
        intermediate_hidden_states, (final_hidden_state, final_cell_state) = self.lstm(embedded)
        output = self.fc(final_hidden_state.squeeze(0))
        return output

In [39]:
model = LSTMModel(len(vocab))

### Selecting GPU 

In [40]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [41]:
model.to(device)

LSTMModel(
  (embedding): Embedding(342, 100)
  (lstm): LSTM(100, 150, batch_first=True)
  (fc): Linear(in_features=150, out_features=342, bias=True)
)

### Set parameters values

In [42]:
epochs = 50
lr = 0.001

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr)

## Training Loop

In [43]:
for epoch in range(epochs):
    total_loss = 0

    for batch_x, batch_y in dataloader:
        
        #setting to gpu
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
    
        optimizer.zero_grad()#clear the gradents
        output = model(batch_x)
        loss = loss_func(output,batch_y)
        loss.backward()
        optimizer.step()
    
        total_loss += loss.item()
    
    print(f"epoch : {epoch}  loss : {loss:.6f}")

epoch : 0  loss : 5.726757
epoch : 1  loss : 5.250511
epoch : 2  loss : 4.809451
epoch : 3  loss : 5.013255
epoch : 4  loss : 4.834305
epoch : 5  loss : 3.661457
epoch : 6  loss : 3.298998
epoch : 7  loss : 2.306647
epoch : 8  loss : 2.352267
epoch : 9  loss : 2.582575
epoch : 10  loss : 2.235348
epoch : 11  loss : 2.222619
epoch : 12  loss : 1.683139
epoch : 13  loss : 1.562171
epoch : 14  loss : 1.301430
epoch : 15  loss : 0.951241
epoch : 16  loss : 0.996923
epoch : 17  loss : 1.026347
epoch : 18  loss : 0.755432
epoch : 19  loss : 0.742291
epoch : 20  loss : 0.773187
epoch : 21  loss : 0.438057
epoch : 22  loss : 0.476288
epoch : 23  loss : 0.440844
epoch : 24  loss : 0.320719
epoch : 25  loss : 0.317488
epoch : 26  loss : 0.368161
epoch : 27  loss : 0.373233
epoch : 28  loss : 0.357020
epoch : 29  loss : 0.340927
epoch : 30  loss : 0.329642
epoch : 31  loss : 0.303904
epoch : 32  loss : 0.546184
epoch : 33  loss : 0.137163
epoch : 34  loss : 0.364670
epoch : 35  loss : 0.284062
ep

## Prediction

In [54]:
def prediction(model, vocab, text):
    tokenized_text = word_tokenize(text.lower())
    numerical_text = text_to_index(tokenized_text,vocab)
    padded_text = torch.tensor([0]*(46 - len(numerical_text)) + numerical_text, dtype=torch.long).unsqueeze(0)
    output = model(padded_text)
    value, index = torch.max(output, dim=1)
    
    pred_text = text + " " + list(vocab.keys())[index]
    return pred_text

In [55]:
predicted_text = prediction(model,vocab,'Virat Kohli holds')
print(predicted_text)

Virat Kohli holds many


In [58]:
no_of_tokens = 10
input_text = "Virat Kohli holds"
print(input_text)

for i in range(no_of_tokens):
    output = prediction(model,vocab,input_text)
    print(output)
    input_text = output

Virat Kohli holds
Virat Kohli holds many
Virat Kohli holds many records
Virat Kohli holds many records including
Virat Kohli holds many records including most
Virat Kohli holds many records including most odi
Virat Kohli holds many records including most odi centuries
Virat Kohli holds many records including most odi centuries ,
Virat Kohli holds many records including most odi centuries , fastest
Virat Kohli holds many records including most odi centuries , fastest to
Virat Kohli holds many records including most odi centuries , fastest to multiple


## Predicting next 3 words

In [59]:
def prediction(model, vocab, text, k=3):
    tokenized_text = word_tokenize(text.lower())
    numerical_text = text_to_index(tokenized_text, vocab)

    padded_text = torch.tensor([0] * (46 - len(numerical_text)) + numerical_text,dtype=torch.long).unsqueeze(0)
    output = model(padded_text)
    probs = torch.softmax(output, dim=1)
    values, indices = torch.topk(probs, k, dim=1)

    predicted_words = []
    for i in range(k):
        word = list(vocab.keys())[indices[0][i]]
        confidence = values[0][i].item()
        predicted_words.append((word, confidence))

    return predicted_words


In [78]:
no_of_tokens = 15
input_text = "Cricket is played"
print(input_text)

for i in range(no_of_tokens):
    output = prediction(model,vocab,input_text)
    print(output)
    
    best_word = output[0][0]
    
    input_text += " " + best_word
    print("\n",input_text)
    

Cricket is played
[('mainly', 0.5017523169517517), ('in', 0.4368796944618225), ('between', 0.014548171311616898)]

 Cricket is played mainly
[('in', 0.9833337664604187), ('mainly', 0.0034382883459329605), ('formats', 0.001698858686722815)]

 Cricket is played mainly in
[('three', 0.8956617116928101), ('over', 0.01637955754995346), ('the', 0.010497527197003365)]

 Cricket is played mainly in three
[('formats', 0.9328184723854065), (':', 0.018405862152576447), ('in', 0.010678092017769814)]

 Cricket is played mainly in three formats
[(':', 0.9385566115379333), ('formats', 0.021165883168578148), ('of', 0.013646485283970833)]

 Cricket is played mainly in three formats :
[('test', 0.9610836505889893), ('cricket', 0.01587207242846489), (':', 0.0084830978885293)]

 Cricket is played mainly in three formats : test
[('cricket', 0.9543183445930481), ('test', 0.010870535857975483), (',', 0.0061530922539532185)]

 Cricket is played mainly in three formats : test cricket
[(',', 0.9645594954490662)